In [1]:
from deeprobust.graph.data import Dataset
from scapin import Scapin
import numpy as np
import torch
import argparse
import utils


parser = argparse.ArgumentParser()
parser.add_argument("--seed", default=0, type=int)
parser.add_argument("--device", default="cuda:0")
parser.add_argument("--dataset", default="citeseer")

args, unknown = parser.parse_known_args()

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.device != 'cpu':
    torch.cuda.manual_seed(args.seed)

data = Dataset(root='/tmp/', name=args.dataset, setting='nettack')

worker = Scapin(adj=data.adj, features=np.asarray(data.features.todense()), labels=data.labels,
                split_idx={"train": data.idx_train,
                           "valid": data.idx_val,
                           "test": data.idx_test},
                _ptb_rate=0.05, _device="cuda:0", _th=0.085)

adj_res = worker.attack()

Loading citeseer dataset...
Selecting 1 largest connected components


In [2]:
print("clean:")
utils.test(utils.sparse_mx_to_torch_sparse_tensor(data.adj), worker.features_torch,
     worker.labels_torch, worker.train_idx, worker.valid_idx, worker.test_idx, args.device)

clean:
Test set results: loss= 1.2378 accuracy= 0.7115


0.7114928909952607

In [3]:
print("scapin:")
utils.test(utils.sparse_mx_to_torch_sparse_tensor(adj_res), worker.features_torch,
     worker.labels_torch, worker.train_idx, worker.valid_idx, worker.test_idx, args.device)

scapin:
Test set results: loss= 1.4977 accuracy= 0.6114


0.6113744075829384